In [ ]:
#!conda install ephem --yes

In [ ]:
# runs in 8 seconds
import ephem
import numpy
import pandas
from SciServer import SciDrive
from astropy import units
from astropy.coordinates import SkyCoord
from SciServer import CasJobs
print('Done')

In [ ]:
# runs in 24 seconds
#SciDrive.directoryList('mpc')
mpc = SciDrive.download('mpc/MPCORB.DAT', localFilePath='mpcorb.dat')
print('Done')

In [ ]:
# runs in 1m 15s
df = pandas.read_fwf('mpcorb.dat')
df = df.set_index('desig')
print('Data read')
df = df.rename(columns = {'epc':'epoch', 'meanAn': 'meanAnomaly', 'perihel':'periheleonArgument'})
df = df.rename(columns = {'nodAL':'nodeAscendingLongitude', 'incl': 'inclination'})
df = df.rename(columns = {'motDail': 'motionDailyMean', 'z':'uncertainty', 'ref': 'reference'})
df = df.rename(columns = {'nOb': 'nObservations', 'nP':'nOppositions', 'arcL':'arcLength'})
df = df.rename(columns = {'rms':'rmsResidual', 'p': 'perturbersCoarse', 'pp':'perturbersPrecise'})
df = df.rename(columns = {'comp':'computerName', 'flg':'flags', 'lastO':'lastObservationDate'})
print('Columns renamed')
print('Done')

In [ ]:
#runs in 23 sec
df['lastObservationDate'] = df['lastObservationDate'].astype(str)
print('Dates stringed')
df['lastObservationDate'] = df['lastObservationDate'].apply(lambda x: x[0:-2])
print('Dates parsed')
df['lastObservationDate'] = pandas.to_datetime(df['lastObservationDate'], format='%Y%m%d', errors='coerce')
print('Dates dated')
df['lastObservationYear'] = df['lastObservationDate'].apply(lambda x: x.year)
print('Years yeared')

df['orbit_type'] = 'e'
print('Orbit type set to e')

xephem_column_order = ['name', 'orbit_type', 'inclination']
xephem_column_order += ['nodeAscendingLongitude', 'periheleonArgument', 'a']
xephem_column_order += ['motionDailyMean', 'e', 'meanAnomaly']
xephem_column_order += ['epoch', 'lastObservationYear', 'h', 'g']
xephem_df = df[xephem_column_order]
print('xephem xephemed')
print('Done')

In [ ]:
# runs in 1h 20min

# structure of .epb file (what we need to convert to):
# names, {f|e|h|p|E|P}, incli, long-ascending-node, arg-of-perihel, a (mean dist), 
##  mean-daily-motion, eccentricity, mean-anomaly, epoch, equinox-year, mag1, mag2, ang-size

'''
xephem_column_order = ['name', 'orbit_type', 'inclination']
xephem_column_order += ['nodeAscendingLongitude', 'periheleonArgument', 'a']
xephem_column_order += ['motionDailyMean', 'e', 'meanAnomaly']
xephem_column_order += ['epoch', 'lastObservationYear', 'h', 'g']

xephem_df = df[xephem_column_order]
'''

#aline = "C/2002 Y1 (Juels-Holvorcem),e,103.7816,166.2194,128.8232,242.5695,0.0002609,0.99705756,0.0000,04/13.2508/2003,2000,g  6.5,4.0"
#print(aline)
#print('\n')

with open('orbits.csv', 'w') as f:
    for i in range(0,len(df)):
#    for i in range(0,101):
        if (numpy.mod(i,10000) == 0):
            print('Writing data for asteroid {:,.0f}...'.format(i))
        thisline = ""
        for k in range(0,11):
            thisline += str(xephem_df.iloc[i].values[k]) + ","
        thisline += 'H '+str(xephem_df.iloc[i].values[11]) + ","
        thisline += str(xephem_df.iloc[i].values[12])
        thisline += '\n'
#        print(thisline)
        f.write(thisline)
print('Done')

In [ ]:
# runs in about 1 min
ra_list = []
dec_list = []
cnt = 1
for line in open('orbits.csv'):
    yh = ephem.readdb(line)
    if (numpy.mod(cnt,10000) == 0):
        print('Read data for asteroid {:}'.format(cnt))
    yh.compute('2017/9/1')
    if (numpy.mod(cnt,10000) == 0):
        print('Calculated ra/dec for asteroid {:,.0f}'.format(cnt))
    ra_list.append(str(yh.ra))
    dec_list.append(str(yh.dec))
    cnt = cnt + 1

print('Done')

In [ ]:
# runs in about 2 h 30 min
coord_deg = numpy.empty((len(df),2))
coord_deg[:] = numpy.nan

for i in range(0, len(ra_list)):
    thestr = ra_list[i] + ' ' + dec_list[i]
#    print(thestr)
    a = SkyCoord(thestr, unit='hourangle, deg')
#    print(a)
#    print(a.ra.deg, a.dec.deg)
    coord_deg[i,0] = a.ra.deg
    coord_deg[i,1] = a.dec.deg
    if (numpy.mod(i, 10000) == 0):
        print('Converted coordinates for asteroid {:,.0f}'.format(i))
#    df.loc[i, 'ra'] = a.ra.deg
#    df.loc[i, 'dec'] = a.dec.deg
#    print('df updated')
#    enya1927_df.loc[i, 'game_in_season'] = thisgame
print(coord_deg)
df['ra'] = coord_deg[:,0].tolist()
df['dec'] = coord_deg[:,1].tolist()
print('Done')
#df.head(100)

In [ ]:
df['name'] = df['name'].replace("'","''")
print('string replaced')

In [ ]:
# runs in 2 minutes
haswritten = CasJobs.uploadPandasDataFrameToTable(df, 'mpcorb','MyDB')
print(haswritten)
#df.columns
#df['epoch'] = df['epoch'].to_string()
#df['epoch'] = df['epoch'].apply(lambda x: str(x))
#print(df.dtypes)

#df['epoch'].head(10)

In [ ]:
df.shape